In [34]:
import numpy as np
import pandas as pd
import time
import sys
import random
from helper_functions_assignment2 import*

In [35]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "test_instances/" #"inst_competition/" #"inst_tuning/" # 
#problem_instance = "heur002_n_100_m_3274" #"heur002_n_100_m_3274" 
problem_instance = "heur001_n_10_m_31" 
#problem_instance = "heur014_n_300_m_14191" 
problem_instance = "heur039_n_361_m_13593"
path = "data/"+instance_type + problem_instance +".txt"

In [36]:
node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)
print(s)
print(n)
print(m)

2
300
14191


In [37]:
plex_assignment = np.random.choice(5, n) #np.range(n) of size n (second is the size)
solution = assignment_to_solution(plex_assignment, node_impact_orig, 
                                    node_degree_orig, edge_weights, edge_assignment_orig, s)

#old = solution.edge_assignment
#print(node_impact_orig)
#print(np.where(old == 0))

#plex_assignment, edge_assignment, removed = remove_random_node(plex_assignment, solution.edge_assignment, n, node_impact_orig, node_degree_orig, edge_weights)

#with np.printoptions(threshold=np.inf):
#    print(removed)
#    print(np.where(edge_assignment == 0))

In [38]:
random.seed(4)
destroy_methods = [remove_random_node, remove_highest_cost_node, remove_most_edges_node, 
                   remove_smallest_splex, remove_largest_splex]
repair_methods = [add_to_largest_splex, add_to_smallest_splex, add_new_splex, add_to_random_splex]

destroy_weights = np.ones(len(destroy_methods))
repair_weights = np.ones(len(repair_methods))

# Parameters:
min_weight = 0.05
reaction_factor = 0.1
iterations_per_phase = 10

def update_weight(old_weight, num_applications, num_successes):
    if num_applications == 0:
        return old_weight
    new_weight = old_weight * (1-reaction_factor) + reaction_factor * (num_successes / num_applications)
    return max(new_weight, min_weight) # Make sure weight never equals 0

def get_node_properties(edge_assignment, edge_weights, plex_assignment, n):
    calc_node_impact = np.zeros(n)
    calc_node_degree = np.zeros(n)


    #print("edge_weights " + str(edge_weights))
    #print("edge_assigments " + str(edge_assignment))
    actual_weights = np.zeros(len(edge_assignment))

    #print(str(len(edge_weights)) + " " + str(len(edge_assignment)) + " " + str(len(actual_weights)))
    # Get actual positive weights
    actual_weights[edge_weights < 0] = edge_weights[edge_weights < 0] * (edge_assignment[edge_weights < 0] -1)
    actual_weights[edge_weights > 0] = edge_weights[edge_weights > 0] * edge_assignment[edge_weights > 0]
    #print("product " + str(actual_weights))

    for i in range(n):
        possible_edge_indices = np.fromiter(((get_edge_index(i+1, b+1, n), b)
                                            for b in range(n)
                                            if i != b),
                                            np.dtype((plex_assignment.dtype, 2)))

        #print("ea " + str(edge_assignment[possible_edge_indices]))
        #print("aw " + str(actual_weights[possible_edge_indices]))

        calc_node_impact[i] = actual_weights[possible_edge_indices[0]].sum()
        calc_node_degree[i] = edge_assignment[possible_edge_indices[0]].sum()

        for ind in possible_edge_indices:
            if plex_assignment[i] != plex_assignment[ind[1]] and edge_assignment[ind[0]] == 1:
                raise TypeError("Hey! " + str(plex_assignment[i]) + " != " + str(plex_assignment[ind[1]]) + "(" + str(ind[0]) + ")")

    return calc_node_degree, calc_node_impact

# get all infos from instance
node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)
node_impact = node_impact_orig.copy()
node_degree = node_degree_orig.copy()
edge_assignment = edge_assignment_orig.copy()

plex_assignment = np.random.choice(3, n) #np.range(n) of size n (second is the size)
print("initial plex assignment", plex_assignment)

# correct node and edge infos to make a valid plex
repair_solution(node_impact, node_degree, plex_assignment, edge_weights, edge_assignment, s)

print(edge_assignment_orig)
print(node_impact)
print("node degree", node_degree)

#edge_assignment = solution.edge_assignment
print(edge_assignment)
current_score = sum(node_impact)/2

for i in range(100):
    destroy_prob = destroy_weights/destroy_weights.sum()
    repair_prob = repair_weights/repair_weights.sum()

    print(destroy_prob)
    print(repair_prob)

    destroy_applications = np.zeros(len(destroy_methods))
    destroy_successes = np.zeros(len(destroy_methods))
    repair_applications = np.zeros(len(repair_methods))
    repair_successes = np.zeros(len(repair_methods))

    for j in range(iterations_per_phase):
        destroy_idx = np.random.choice(len(destroy_methods), p=destroy_prob)
        repair_idx = np.random.choice(len(repair_methods), p=repair_prob)
        
        destroy_applications[destroy_idx] += 1
        repair_applications[repair_idx] += 1

        plex_assignment_new = plex_assignment.copy()
        edge_assignment_new = edge_assignment.copy()
        node_impact_new = node_impact.copy()
        node_degree_new = node_degree.copy()
        
        #print(destroy_methods[destroy_idx])
        _, _, removed_nodes = destroy_methods[destroy_idx](plex_assignment_new, edge_assignment_new, n, node_impact_new, node_degree_new, edge_weights)
        
        #actual_degree, actual_impact = get_node_properties(edge_assignment_new, edge_weights, plex_assignment_new, n)
        
        #if not np.array_equal(node_degree_new, actual_degree):
        #    print("degrees not equal")
        #    print(np.flatnonzero((~np.equal(node_degree_new, actual_degree)).astype(int)))
        #    print(node_degree_new)
        #    print(actual_degree)
        #    print()
            
        #if not np.array_equal(node_impact_new, actual_impact):
        #    print("impact not equal")
        #    print(np.flatnonzero((~np.equal(node_impact_new, actual_impact)).astype(int)))
        #    print(node_impact_new)
        #    print(actual_impact)
        #    print()
        
        #print(repair_methods[repair_idx])
        _, _ = repair_methods[repair_idx](plex_assignment_new, edge_assignment_new, n, removed_nodes, node_impact_new, node_degree_new, edge_weights, s)
        
        #actual_degree, actual_impact = get_node_properties(edge_assignment_new, edge_weights, plex_assignment_new, n)
        
        #if not np.array_equal(node_degree_new, actual_degree):
        #    print("degrees not equal")
        #    print(np.flatnonzero((~np.equal(node_degree_new, actual_degree)).astype(int)))
        #    print(node_degree_new)
        #    print(actual_degree)
        #    print()
            
        #if not np.array_equal(node_impact_new, actual_impact):
        #    print("impact not equal")
        #    print(np.flatnonzero((~np.equal(node_impact_new, actual_impact)).astype(int)))
        #    print(node_impact_new)
        #    print(actual_impact)
        #    print()

        new_score = sum(node_impact_new)/2
        accept_solution = False
        if new_score < current_score:
            accept_solution = True
            destroy_successes[destroy_idx] += 1
            repair_successes[repair_idx] += 1
        
            current_score = new_score
            plex_assignment = plex_assignment_new
            edge_assignment = edge_assignment_new
            node_impact = node_impact_new
            node_degree = node_degree_new          

    for j in range(0, len(destroy_methods)):
        destroy_weights[j] = update_weight(destroy_weights[j], destroy_applications[j], destroy_successes[j])
        
    for j in range(0, len(repair_methods)):
        repair_weights[j] = update_weight(repair_weights[j], repair_applications[j], repair_successes[j])
    
    print("iter")

initial plex assignment [1 2 0 2 0 2 2 2 0 2 2 2 1 0 1 2 0 2 0 0 2 1 0 2 1 1 1 0 1 1 2 1 2 2 0 2 0
 2 2 0 2 2 0 2 1 2 0 2 1 1 1 2 0 1 2 1 0 0 2 2 2 0 2 2 1 0 1 2 2 2 1 0 2 0
 1 2 0 0 1 0 0 0 0 2 2 1 2 2 2 1 2 2 2 2 1 0 2 0 2 0 1 1 2 0 1 1 1 2 1 1 1
 2 1 0 0 1 2 1 2 0 2 0 2 0 1 0 0 1 2 0 1 1 1 0 1 0 1 1 2 1 1 2 0 0 1 2 1 0
 2 0 2 1 2 1 1 0 2 1 1 0 0 2 2 2 1 2 0 1 1 2 0 1 1 1 1 2 0 2 1 2 0 0 2 0 2
 2 2 0 2 2 2 1 0 0 0 0 2 0 2 0 1 0 2 1 1 0 1 0 1 2 1 1 0 2 2 0 0 2 2 1 2 2
 1 2 0 0 1 1 0 1 1 0 1 1 0 0 0 2 1 2 0 1 0 1 1 0 0 0 2 1 2 0 2 1 2 2 2 1 2
 0 1 0 0 2 0 1 2 1 1 0 1 0 0 0 2 2 0 0 1 1 2 0 0 1 1 2 0 0 1 2 1 0 1 1 0 1
 1 2 2 2]
[0 0 0 ... 0 0 0]
[511. 410. 420. 385. 387. 402. 388. 406. 508. 450. 319. 347. 383. 388.
 339. 362. 428. 351. 296. 357. 360. 310. 372. 381. 381. 394. 318. 346.
 357. 372. 373. 416. 339. 396. 413. 437. 428. 347. 385. 428. 365. 377.
 494. 405. 432. 405. 416. 351. 437. 412. 462. 439. 417. 447. 444. 486.
 441. 415. 367. 425. 385. 432. 373. 405. 376. 444. 460. 396. 446

In [39]:
#get_edge_nodes(43, n)
get_edge_index(5, 8, n)

1192